In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 38.9 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
import re

# 데이터 로드
data = pd.read_excel('/content/drive/MyDrive/[텍스트분석] 팀 프로젝트/dataset/article_df_final.xlsx', engine='openpyxl')

# 전처리: 한글 형태소 분석 및 특수문자 제거
okt = Okt()
data['main'] = data['main'].apply(lambda x: ' '.join(okt.morphs(re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣\s]", "", x))))


# TF-IDF 벡터화
cv = CountVectorizer()

X = cv.fit_transform(data['main'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(X)

# 추천 함수 정의
def get_recommendations(idx, cosine_sim=cosine_sim):
    # 해당 기사와 모든 기사와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 기사들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 20개의 기사를 받아온다.
    sim_scores = sim_scores[1:21]

    # 가장 유사한 20개의 기사의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 20개의 기사내용과 유사도를 리턴한다.
    similar_articles = data[['main', 'section']].iloc[movie_indices]
    similar_articles['similarity'] = [idx[1] for idx in sim_scores]

    return similar_articles





# 추천 기사 호출
goal = int(input("찾으려는 기사의 인덱스를 입력하세요: "))
print("해당 기사: ")
print(data.loc[goal])
print("\n유사 기사:")
ans = get_recommendations(goal)
ans
